I am running `trim_galore` (<https://github.com/FelixKrueger/TrimGalore>) to automatically detect and remove adapters (and very short reads). 

In [1]:
# set important data in the config file first!
source("config.R")

Auto-refreshing stale OAuth token.

v Reading from Status of mouse sequencing.

v Range ''All founder files''.


Attaching package: 'tidytable'


The following objects are masked from 'package:stats':

    dt, filter, lag


The following object is masked from 'package:base':

    %in%




In [2]:
source("../../extra-R-functions.R")

In [3]:
library(stringdist)


Attaching package: 'stringdist'


The following object is masked from 'package:tidytable':

    extract




In [4]:
fastq_files=list.files(path=folder_for_sequences,pattern="fq.gz$")
fastq_files=c(fastq_files,list.files(path=folder_for_sequences,pattern="fastq.gz$"))
fastq_files=sort(fastq_files)
head(fastq_files)

[1] "18163FL-02-01-01_S1_L006_R1_001.fastq.gz"
[2] "18163FL-02-01-01_S1_L006_R2_001.fastq.gz"
[3] "18163FL-02-01-04_S4_L006_R1_001.fastq.gz"
[4] "18163FL-02-01-04_S4_L006_R2_001.fastq.gz"
[5] "18163FL-02-01-06_S6_L006_R1_001.fastq.gz"
[6] "18163FL-02-01-06_S6_L006_R2_001.fastq.gz"

In [5]:
length(fastq_files)

[1] 10

In [6]:
HaveFilesBeenTrimmed=function(){
    # see if files have been trimmed
    trimmed_files_expected_1=paste0(str_replace(str_replace(fastq_files,fixed(".fq.gz"),
                                                ""),fixed(".fastq.gz"),""),"_val_1.fq.gz")
    trimmed_files_expected_2=paste0(str_replace(str_replace(fastq_files,fixed(".fq.gz"),
                                                ""),fixed(".fastq.gz"),""),"_val_2.fq.gz")

    if(!dir.exists(trim_dir)){
        dir.create(trim_dir)
    }
    trimmed_files=list.files(path=trim_dir,pattern="fq.gz$")

    not_yet_trimmed=!(trimmed_files_expected_1%in%trimmed_files) & !(trimmed_files_expected_2 %in% trimmed_files)
    
    return(not_yet_trimmed)
}

In [7]:
# see if files have been trimmed
not_yet_trimmed=which(HaveFilesBeenTrimmed())

fastq_files=fastq_files[not_yet_trimmed]

In [8]:
# test if pairs make sense

#calculate Levenshtein distance between two strings
if(length(fastq_files)>0){
    

    stringdist(fastq_files[seq(from=1,to=length(fastq_files)-1,by=2)], fastq_files[seq(from=2,to=length(fastq_files),by=2)], method = "lv")
}
# 1 is obviously the best case but 3 means "fwd" vs "rev"

[1] 1 1 1 1 1

In [9]:
fastq_files

[1] "18163FL-02-01-01_S1_L006_R1_001.fastq.gz"
 [2] "18163FL-02-01-01_S1_L006_R2_001.fastq.gz"
 [3] "18163FL-02-01-04_S4_L006_R1_001.fastq.gz"
 [4] "18163FL-02-01-04_S4_L006_R2_001.fastq.gz"
 [5] "18163FL-02-01-06_S6_L006_R1_001.fastq.gz"
 [6] "18163FL-02-01-06_S6_L006_R2_001.fastq.gz"
 [7] "18163FL-02-01-07_S7_L006_R1_001.fastq.gz"
 [8] "18163FL-02-01-07_S7_L006_R2_001.fastq.gz"
 [9] "18163FL-08-01-02_S2_R1_001.fastq.gz"     
[10] "18163FL-08-01-02_S2_R2_001.fastq.gz"

In [11]:
jobname="trim_galore"
if(length(fastq_files)>0) { 
    for(s in seq(from=1,to=length(fastq_files)-1,by=2)){
        cmd=paste0("cd ", folder_for_sequences, " && ",
                   "Rscript ",jnr_general_scripts_dir,"FixTooShortFastq.R ",paste(fastq_files[c(s,s+1)], collapse = " ")," ",jnr_general_scripts_dir," && ", 
                   ". ",Barn_Mice_dir,"activateEnv.sh ", env_mapping_etc," && trim_galore --paired -o trim_galore -j 1 ",paste(fastq_files[c(s,s+1)], collapse = " ")) 
        execute_cmd_sbatch(cmd, mem="8gb", cpu="1", time="short", env=env_jupyter, acc=slurm_acc, jobname="trim_galore", activateEnvScript=paste0(Barn_Mice_dir,"activateEnv.sh")) 
        Sys.sleep(30)
    }
}

[1] "sbatch -c 1 --mem=8gb --job-name=trim_galore -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh my_conda4; cd /moto/ziab/users/jr3950/data/genomes/tmp_founders/ && Rscript ~/github/general/FixTooShortFastq.R 18163FL-02-01-01_S1_L006_R1_001.fastq.gz 18163FL-02-01-01_S1_L006_R2_001.fastq.gz ~/github/general/ && . ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116 && trim_galore --paired -o trim_galore -j 1 18163FL-02-01-01_S1_L006_R1_001.fastq.gz 18163FL-02-01-01_S1_L006_R2_001.fastq.gz'"
[1] "Submitted batch job 13323528"
[1] "sbatch -c 1 --mem=8gb --job-name=trim_galore -A ziab -t 11:59:00 --wrap '. ~/ColumbiaProjects/Barn_Mice/activateEnv.sh my_conda4; cd /moto/ziab/users/jr3950/data/genomes/tmp_founders/ && Rscript ~/github/general/FixTooShortFastq.R 18163FL-02-01-04_S4_L006_R1_001.fastq.gz 18163FL-02-01-04_S4_L006_R2_001.fastq.gz ~/github/general/ && . ~/ColumbiaProjects/Barn_Mice/activateEnv.sh samtools-116 && trim_galore --paired -o trim_galore -j

In [ ]:
while(slurm_check_jobs_still_running(columbia_username,jobname)){
                Sys.sleep(60)
            }
fastq_files=list.files(path=folder_for_sequences,pattern="fq.gz$")
fastq_files=c(fastq_files,list.files(path=folder_for_sequences,pattern="fastq.gz$"))
fastq_files=sort(fastq_files)
trimmed=which(!HaveFilesBeenTrimmed())
file.remove(paste0(folder_for_sequences,fastq_files[trimmed]))
file.remove(paste0(folder_for_sequences,fastq_files[trimmed],".gziptest"))